In [6]:
API_KEY = 'y5Pt4OyhvXKRMNgmQUudrQTVp'
ACESS_TOKEN = '1536862091537178624-BRUEJHXCKJkb9EQP4eMunnmAyP6TDT'
API_KEY_SECRET = 'H2bRlfYoQ9appGSqiQvSONvyitEuttt87fPtYlYhlmYtsRzR5A'
ACESS_TOKEN_SECRET = 'Q6vzz1YsRxOhJ1mDHOXVSt1Id687GuwL1l1yhkRm92SQi'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAI4ZdwEAAAAAr2Z9ihJUyjN2Z5aAsiVFH0zt%2FIM%3DtUFLGD7Fo30mYT5hj2IJwVex27bCSKz2ZAgQDssNKs6HssNBD6'

In [5]:
import sys
!{sys.executable} -m pip install pandas

You should consider upgrading via the 'c:\Users\eduar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import sys
!{sys.executable} -m pip install tweepy

You should consider upgrading via the 'c:\Users\eduar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [28]:
import tweepy
import pandas as pd
import os  

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [29]:
def pesquisa_politico(politico):
    data = client.search_recent_tweets(query=politico +' -is:retweet',max_results=100,tweet_fields=['created_at','lang','public_metrics','source','geo'],place_fields=['country','geo','name','country_code'],expansions=['geo.place_id','author_id'])
    return data

In [14]:
def preenche_listas(data,text,id,retweets,replys,likes,author_id,source,lang,created_at,place_id,geo_full_name,geo_x1,geo_x2,geo_y1,geo_y2,geo_id):
    for e in data:
        if isinstance(e,dict):
            for i in e.keys():
                if i == 'places':
                    for l in e[i]:
                        geo_full_name.append(l.full_name)
                        geo_x1.append(l.data['geo']['bbox'][1])
                        geo_x2.append(l.data['geo']['bbox'][0])
                        geo_y1.append(l.data['geo']['bbox'][3])
                        geo_y2.append(l.data['geo']['bbox'][2])
                        geo_id.append(l.id)
        if isinstance(e,list):
            for i in e:
                place_id.append(i.geo)
                created_at.append(i.created_at)
                lang.append(i.lang)
                source.append(i.source)
                author_id.append(i.author_id)
                retweets.append(i.public_metrics['retweet_count'])
                likes.append(i.public_metrics['like_count'])
                replys.append(i.public_metrics['reply_count'])
                id.append(i.id)
                text.append(i.text)

In [19]:
def salvar_100(politico, df_tweet, df_tweet_geo):
    os.makedirs('dados/ultimos 100 tweets', exist_ok=True)  
    df_tweet.to_csv('dados/ultimos 100 tweets/'+politico+'_tweets.csv') 
    df_tweet_geo.to_csv('dados/ultimos 100 tweets/'+politico+'_tweets_locais.csv') 

In [16]:
def criar_csv_100tweets(politico):
    data = pesquisa_politico(politico)
    text = []
    id = []
    retweets = []
    replys = []
    likes = []
    author_id = []
    source = []
    lang = []
    created_at = []
    place_id = []
    geo_full_name = []
    geo_x1 = []
    geo_x2 = []
    geo_y1 = []
    geo_y2 = []
    geo_id = []
    preenche_listas(data,text,id,retweets,replys,likes,author_id,source,lang,created_at,place_id,geo_full_name,geo_x1,geo_x2,geo_y1,geo_y2,geo_id)
    df_tweet = pd.DataFrame(data={'text' : text, 'id' : id, 'retweets' : retweets, 'replys' : replys,'likes' : likes, 'author_id' : author_id, 'source' : source, 'lang' : lang, 'created_at': created_at,'place_id' : place_id})
    df_tweet_geo = pd.DataFrame(data={'geo_full_name' : geo_full_name,'geo_id' : geo_id, 'geo_x1' : geo_x1, 'geo_x2' : geo_x2, 'geo_y1' : geo_y1, 'geo_y2' : geo_y2})
    salvar_100(politico, df_tweet,df_tweet_geo)

In [21]:
pre_candidatos = ['bolsonaro', 
                    'lula', 
                    'ciro', 
                    'simone tebet', 
                    'bivar', 
                    'Luiz Felipe d’Avila', 
                    'janones', 
                    'leonardo pericles', 
                    'pablo maçal', 
                    'vera lucia', 
                    'sofia manzano', 
                    'eymael']

In [20]:
for c in pre_candidatos:
    criar_csv_100tweets(c)

In [35]:
def pesquisa_p_t_count(politico,tema):
    counts = client.get_recent_tweets_count(query=politico+' '+tema+' -is:retweet',granularity='day')
    return counts.data

In [24]:
def salvar_7(politico, df_tweet, tipo):
    os.makedirs('dados/ultimos 7 dias', exist_ok=True)  
    df_tweet.to_csv('dados/ultimos 7 dias/'+tipo+politico+'_por_tema_tweets.csv') 

In [33]:
def criar_csv_7dias(politico, temas, tipo = ''):
    valor_tema_dia = []
    for i in range(len(temas)):
        valor_tema_dia.append([])
        pesquisa = pesquisa_p_t_count(politico,temas[i])
        if i == 0:
            dias = []
        for j in range(len(pesquisa)):
            valor_tema_dia[i].append(pesquisa[j]['tweet_count'])
            if i == 0:
                dias.append(pesquisa[j]['start'])
    datas = dict(zip(temas, valor_tema_dia))
    df_tweet = pd.DataFrame(data=datas,index=dias)
    print(df_tweet)
    salvar_7(politico, df_tweet, tipo)

In [26]:
temas = ['ditadura',
        'reforma trabalhista',
        'feminismo',
        'reforma agrária',
        'corrupção',
        'segurança']

In [36]:
for pre_candidato in pre_candidatos:
    criar_csv_7dias(pre_candidato, temas)

                          ditadura  reforma trabalhista  feminismo  \
2022-06-10T18:24:16.000Z        37                   12          1   
2022-06-11T00:00:00.000Z       152                   12          5   
2022-06-12T00:00:00.000Z       116                   13          3   
2022-06-13T00:00:00.000Z       175                    4          6   
2022-06-14T00:00:00.000Z       132                   19          3   
2022-06-15T00:00:00.000Z       168                   19          2   
2022-06-16T00:00:00.000Z       126                    6          7   
2022-06-17T00:00:00.000Z        84                    4          6   

                          reforma agrária  corrupção  segurança  
2022-06-10T18:24:16.000Z                2        118         82  
2022-06-11T00:00:00.000Z                5        374        134  
2022-06-12T00:00:00.000Z               12        460        195  
2022-06-13T00:00:00.000Z                9        571        237  
2022-06-14T00:00:00.000Z               

In [ ]:
negativos = ['ditadura',
            'reforma trabalhista',
            'feminismo',
            'reforma agrária',
            'corrupção',
            'segurança']

In [ ]:
for pre_candidato in pre_candidatos:
    criar_csv_7dias(pre_candidato, negativos, tipo='neg')

In [ ]:
positivos = ['ditadura',
            'reforma trabalhista',
            'feminismo',
            'reforma agrária',
            'corrupção',
            'segurança']

In [ ]:
for pre_candidato in pre_candidatos:
    criar_csv_7dias(pre_candidato, positivos, tipo='pos')